# Парсинг социальных сетей и регулярные выражения

## Регулярные выражения

Прежде чем мы приступим к самому интересному -- парсингу соц. сетей, давайте научимся регулярным выражениям за 10 минут.

переходим **СЮДА**: https://regexone.com/.

Особым любителям регулярок [сюда](https://alf.nu/RegexGolf).

Обязательно к прочтению дома:

* [Регулярные выражения, пособие для новичков. Часть 1](https://habr.com/ru/post/115825/)

* [Регулярные выражения, пособие для новичков. Часть 2](https://habr.com/ru/post/115436/)

* https://ru.wikipedia.org/wiki/Регулярные_выражения

Загружаем библиотеку для регулярных выражений:

In [38]:
import re

In [40]:
text = "Киркегор - датский философ, богослов и писатель, один из предшественников экзистенциализма. С. Кьеркегор окончил теологический факультет Копенгагенского университета в 1840 году. Степень магистра получил в 1841 году, защитив диссертацию “О понятии иронии, с постоянным обращением к Сократу”, посвященную концепциям иронии у древнегреческих авторов и романтиков. Работы С. Кьеркегора отличаются исключительной психологической точностью и глубиной. Вклад в развистие философии, сделанный Кьеркегаардом. неоценим. Сёрен Киркегаард: немецкое издание Сёрена Киркегаарда. Спецкурс “С. Керкегор и история христианства в XIX в.” посвящен датскому философу Серену Керкегору."

In [63]:
re.findall(r"(К(е|и|ье)ркег(о|аа)рд?а?у?(ом)?\b)", text)

[('Киркегор', 'и', 'о', ''),
 ('Кьеркегор', 'ье', 'о', ''),
 ('Кьеркегора', 'ье', 'о', ''),
 ('Кьеркегаардом', 'ье', 'аа', 'ом'),
 ('Киркегаард', 'и', 'аа', ''),
 ('Киркегаарда', 'и', 'аа', ''),
 ('Керкегор', 'е', 'о', ''),
 ('Керкегору', 'е', 'о', '')]

In [70]:
len("К(е|и|ье)ркег(о|аа)рд?а?у?(ом)?\b")


word = """sjfgergeuygr-\nskhg sjgh"""

In [71]:
re.sub(r"-\n", "", word)

'sjfgergeuygrskhg sjgh'

С регулярными выражениями мы продолжим работу на следующем занятии, после того, как вы прочитаете эти статьи. Поехали к самому интересному!

## VK

In [ ]:
!pip install vk

In [3]:
import json
import vk

In [25]:
import vk
session=vk.AuthSession(app_id='7969632', user_login='+79308038318', user_password='Hjo4DhBcQUQyU')
api = vk.API(session)

2021-10-07 16:44:35,594 vk    mixins:107 ERROR: Authorization error (incorrect password)


VkAuthError: Authorization error (incorrect password)

In [12]:
token = "4db8a6887bdd0a6098ad33049540d5e675a2a6b15943774ccedf5ddc13319def8fe1f845c041c4465a9c9"  # Сервисный ключ доступа
session = vk.Session(access_token=token)  # Авторизация
vk_api = vk.API(session)

In [34]:
import vk
session = vk.Session()
api = vk.API(session)
api.users.get(user_ids=1)

VkAPIError: 5. User authorization failed: no access_token passed.. request_params = {'method': 'users.get', 'oauth': '1', 'user_ids': '1'}

In [37]:
import vk_api

vk_session = vk_api.VkApi('+79308038318', 'Hjo4DhBcQUQyU')
vk_session.auth()

vk = vk_session.get_api()

print(vk.wall.post(message='Hello world!'))

AuthError: No handler for two-factor authentication

In [32]:
import requests
class VKAuth(object):

    def __init__(self, permissions, app_id, api_v, email=None, pswd=None, two_factor_auth=False, security_code=None, auto_access=True):
        """
        Args:
            permissions: list of Strings with permissions to get from API
            app_id: (String) vk app id that one can get from vk.com
            api_v: (String) vk API version
        """

        self.session        = requests.Session()
        self.form_parser    = FormParser()
        self.user_id        = None
        self.access_token   = None
        self.response       = None

        self.permissions    = permissions
        self.api_v          = api_v
        self.app_id         = app_id
        self.two_factor_auth= two_factor_auth
        self.security_code  = security_code
        self.email          = email
        self.pswd           = pswd
        self.auto_access    = auto_access

        if security_code != None and two_factor_auth == False:
            raise RuntimeError('Security code provided for non-two-factor authorization')
    
    def authorize(self):
        api_auth_url = 'https://oauth.vk.com/authorize'
        app_id = self.app_id
        permissions = self.permissions
        redirect_uri = 'https://oauth.vk.com/blank.html'
        display = 'wap'
        api_version = self.api_v

        auth_url_template = '{0}?client_id={1}&scope={2}&redirect_uri={3}&display={4}&v={5}&response_type=token'
        auth_url = auth_url_template.format(api_auth_url, app_id, ','.join(permissions), redirect_uri, display, api_version)

        self.response = self.session.get(auth_url)

        # look for <form> element in response html and parse it
        if not self._parse_form():
            raise RuntimeError('No <form> element found. Please, check url address')
        else:
            # try to log in with email and password (stored or expected to be entered)
            while not self._log_in():
                pass;

            # handling two-factor authentication
            # expecting a security code to enter here
            if self.two_factor_auth:
                self._two_fact_auth()
    
    def _log_in(self):

        if self.email == None:
            self.email = ''
            while self.email.strip() == '':
                self.email = input('Enter an email to log in: ')

        if self.pswd == None:
            self.pswd = ''
            while self.pswd.strip() == '':
                self.pswd = getpass.getpass('Enter the password: ')

        self._submit_form({'email': self.email, 'pass': self.pswd})
        if not self._parse_form():
            raise RuntimeError('No <form> element found. Please, check url address')

        # if wrong email or password
        if 'pass' in self.form_parser.params:
            print('Wrong email or password')
            self.email = None
            self.pswd = None
            return False
        elif 'code' in self.form_parser.params and not self.two_factor_auth:
            raise RuntimeError('Two-factor authentication expected from VK.\nChange `two_factor_auth` to `True` and provide a security code.')
        else:
            return True
        
    def _two_fact_auth(self):

        prefix = 'https://m.vk.com'
        if prefix not in self.form_parser.url:
            self.form_parser.url = prefix + self.form_parser.url

        if self.security_code == None:
            self.security_code = input('Enter security code for two-factor authentication: ')

        self._submit_form({'code': self.security_code})

        if not self._parse_form():
            raise RuntimeError('No <form> element found. Please, check url address')
    
    def _allow_access(self):
        parser = self.form_parser

        if 'submit_allow_access' in parser.params and 'grant_access' in parser.url:
            if not self.auto_access:
                answer = ''
                msg =   'Application needs access to the following details in your profile:\n' + \
                        str(self.permissions) + '\n' + \
                        'Allow it to use them? (yes or no)'

                attempts = 5
                while answer not in ['yes', 'no'] and attempts > 0:
                    answer = input(msg).lower().strip()
                    attempts-=1

                if answer == 'no' or attempts == 0:
                    self.form_parser.url = self.form_parser.denial_url
                    print('Access denied')

            self._submit_form({})
    
    def _get_params(self):
        try:
            params = self.response.url.split('#')[1].split('&')
            self.access_token = params[0].split('=')[1]
            self.user_id = params[2].split('=')[1]
        except IndexError(e):
            print(e)
            print('Coudln\'t fetch token')

In [35]:
vk = VKAuth(['photos'], '7969632', '5.52')
vk.auth()

access_token = vk.get_token()
user_id = vk.get_user_id()

NameError: name 'FormParser' is not defined

Никто, естественно, не учит методы наизусть. Всегда и везде есть (ну или должна быть) документация https://vk.com/dev/methods

Мы разберем только основные методы, если нужно что-то нестандартное, то стоит сходить в документацию

#### Найти пользователя по id:

In [13]:
people = vk_api.users.get(user_ids=[1, 2, 'lilas_pourpre'], fields=["photo_id", "sex", "bdate", "city", "country", "home_town"], v=5.124)

VkAPIError: 5. User authorization failed: access_token was given to another ip address.. request_params = {'method': 'users.get', 'oauth': '1', 'user_ids': '1,2,lilas_pourpre', 'fields': 'photo_id,sex,bdate,city,country,home_town', 'v': '5.124'}

In [9]:
for person in people:
    print(person['id'], person['last_name'], person['first_name'])

NameError: name 'people' is not defined

#### Найти участников сообщества:

In [18]:
def get_members(groupid, fields=('sex', 'bdate', 'city', 'country', 'domain')):
    first = vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92)  # Первое выполнение метода
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 1000  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    for i in range(1, count+1):  
        data = data + vk_api.groups.getMembers(group_id=groupid, fields=fields, v=5.92, offset=i*1000)["items"]
        
    return data

In [19]:
sbp4_members = get_members("sbp4band")
len(sbp4_members)

104637

In [20]:
sbp4_members[:10]

[{'id': 10,
  'first_name': 'Alexander',
  'last_name': 'Kuznetsov',
  'is_closed': False,
  'can_access_closed': True,
  'sex': 2,
  'domain': 'kuzya',
  'city': {'id': 2, 'title': 'Saint Petersburg'},
  'country': {'id': 1, 'title': 'Russia'}},
 {'id': 172,
  'first_name': 'Alexey',
  'last_name': 'Gorodetsky',
  'is_closed': True,
  'can_access_closed': False,
  'sex': 2,
  'domain': 'g_r_d',
  'bdate': '22.3.1985',
  'city': {'id': 2, 'title': 'Saint Petersburg'},
  'country': {'id': 1, 'title': 'Russia'}},
 {'id': 201,
  'first_name': 'Ignaty',
  'last_name': 'Dyakov',
  'is_closed': False,
  'can_access_closed': True,
  'sex': 2,
  'domain': 'id201',
  'bdate': '3.10.1983',
  'city': {'id': 2, 'title': 'Saint Petersburg'},
  'country': {'id': 1, 'title': 'Russia'}},
 {'id': 272,
  'first_name': 'Pavel',
  'last_name': 'Morzhov',
  'is_closed': False,
  'can_access_closed': True,
  'sex': 2,
  'domain': 'id272',
  'bdate': '23.1.1986',
  'city': {'id': 2, 'title': 'Saint Petersbur

#### Выгрузить посты из сообщества:

In [21]:
def get_posts(owner_id):
    first = vk_api.wall.get(domain=owner_id, count=100, filter="owner", v=5.124)
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 100  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    for i in range(1, count+1):  
        data += vk_api.wall.get(domain=owner_id, v=5.124, filter="owner", count=100, offset=i*100)["items"]
    return [i['text'] for i in data]

In [22]:
sbp4_posts = get_posts("sbp4band")
len(sbp4_posts)

1109

In [25]:
for post in sbp4_posts[:10]:
    print(post)

Новый альбом! СБПЧ “Всё равно” (2020)! 
Слушайте, где хотите: https://band.link/vseravno_sbp4
День ушел искать рассвет! Мы записали кавер на песню Кристины Орбакайте «Позови меня». Слушайте его в специальном плейлисте #ОсвежиМузыкуTicTac в Spotify! 
Ссылка: https://open.spotify.com/playlist/60G7SYsgO9RxVsAYoIoF5B
#ad
13 сентября будем ходить ходуном. Прямо во дворе Пауэрхауса. Всё, как обычно, только соскучившись. Новые песни, старые песни. Прыжки и кувырки. Радость и смех. 
Такое надо в корне пресекать!
Билеты: https://sbp4band.ticketscloud.org

Пожалуйста, планируйте приобретение билетов заранее. Высока вероятность, что продажа на входе осуществляться не будет
Эй, вы не представляете! 
Что вы пропускаете!
Завтра! Новый альбом СБПЧ "Всё равно"!
Я подумал, вы под грибами
27 августа выходит новый альбом СБПЧ «Всё равно».
27 августа выйдет наш новый альбом "Всё равно". А пока вы можете оформить предзаказ. И послушать последнюю песню — "Холодный день". 
http://band.link/SBP4_VSERAVNO
Чере

#### Поиск пабликов по запросу:

In [ ]:
import time

In [ ]:
# https://vkhost.github.io/

personal_token = ""
session = vk.Session(access_token=personal_token)  # Авторизация
vk_api = vk.API(session)

Обратите внимание — даже при использовании параметра offset для получения информации доступны только первые 1000 результатов.

In [ ]:
def get_groups(query):
    first = vk_api.groups.search(q=query, count=1000, type="group", v=5.124)
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"]
    return [(i['name'], i['id']) for i in data]

In [ ]:
found_groups = get_groups("hip-hop")
len(found_groups)

Великолепно. Мы собрали информацию по подписчикам, выгрузили все посты паблика и умеем искать паблики по запросу.

Теперь наша задача -- почистить данные с помощью регулярок и сохранить в файлы.

**Немного про ваш проект:** предполагается, что вы таким образом (или вручную) ищете и находите нужные вам паблики, далее для каждого паблика выгружаете нужную информацию: например, подписчиков (смотрите, из какого они города, их пол и возраст, строите графики) и посты (извлекаете информацию о концертах или об альбомах, клипах, и т.д.)

#### 1. get_groups

Для начала разберемся с найденным списком групп, сохраним его в файл:

In [ ]:
# our code here

Теперь вы всегда можете его заново открыть и использовать в дальнейшем

#### 2. get_members

Теперь разберемся с подписчиками

In [ ]:
len(sbp4_members)

In [ ]:
sbp4_members[0]

In [ ]:
# our code here

#### 3. get_posts

и, наконец, с постами

In [ ]:
from pymystem3 import Mystem

In [ ]:
# your regexps below
http_pattern = re.compile("your regexp here")
user_tag_pattern = re.compile("your regexp here")

In [ ]:
def clean_text(text):
    """
    удалить ссылки, удалить переводы строк, удалить отметки пользователей
    """
    text = re.sub(http_pattern, "", text)
    text = re.sub(user_tag_pattern, "", text)
    text = # your code here (удалить переводы строк)
    return text

In [ ]:
def lemmatize_text(text):
    m = Mystem()
    lemmas = m.lemmatize(text)
    lemmatized_text = ''.join(lemmas)
    return lemmatized_text

In [ ]:
for post in sbp4_posts:
    cleaned = clean_text(post)
    print(cleaned)
    lemmatized = lemmatize_text(cleaned)
    print(lemmatized)
    break

## Twitter

In [ ]:
import requests
from requests_oauthlib import OAuth1
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'

API_KEY=""
API_SECRET=""
ACCESS_TOKEN=""
ACCESS_TOKEN_SECRET=""

auth = OAuth1(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
requests.get(url, auth=auth)

In [ ]:
account_list = ["realDonaldTrump"]

Twitter API explicitly says that you cannot retrieve more then 200 tweets from a user per request so this is not a limitation of this particular library you're using but a limitation imposed by Twitter. But keep in mind that this is per request, if you will send multiple requests you can get up to 3200 tweets in total. But this still means that you can't get all tweets, only 3200.

In [ ]:
r = requests.get('https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=realDonaldTrump&count=200', auth=auth).json()
for i in r:
    print(i['text'])